In [86]:
import astropy
from astropy.io import fits
from astropy.io import ascii
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import odr
from matplotlib.pyplot import figure

In [87]:
#APOKASC file & file variables
hdul = fits.open(r"/Users/amber3/Desktop/SSI/APOKASC_cat_v6.7.2.fits")
#hdul.info()
hdu = hdul[1]
data = hdu.data
#print(np.shape(data))
KIC_A = hdu.data['KEPLER_INT'] 
Logg_A = hdu.data['DR16_LOGG_COR']
Logg_err = hdu.data['DR16_LOGG_COR_ERR']
Teff_A = hdu.data['DR16_TEFF_COR']
Teff_err = hdu.data['DR16_TEFF_COR_ERR']
FeH_A = hdu.data['DR16_M_H_COR']
FeH_A_err = hdu.data['Dr16_M_H_COR_ERR']
#print(Logg_A)
FeH_err = hdu.data['DR16_M_H_COR_ERR']
R_APO3 = hdu.data['APOKASC3P_RADIUS']
R_APO3_err = hdu.data['APOKASC3P_RADIUS_ERR']
R_Gaia = hdu.data['GAIA_RADIUS'] #no error on this radius
M_APO3 = hdu.data['APOKASC3P_MASS']
M_APO3_err = hdu.data['APOKASC3P_MASS_ERR']

In [88]:
#there are some empty entries in error and radius, so flags are put in as -42 and only pos entries are taken
R_data = ascii.read('/Users/amber3/Desktop/gaia_radii_apokasc3 .dat', delimiter = ',') #data_Splitter = 'KIC') #fill_values = [('-42.0', '', '-42.0')])
KIC_Gaia = R_data['KIC']
R_Gaia = 1.0/R_data['inv_r_gaia_K_MIST']
#R_Gaia_err = 1.0/R_data['inv_r_gaia_K_MIST_err']
R_Gaia_err_per = R_data['inv_r_gaia_K_MIST_err']/R_data['inv_r_gaia_K_MIST'] 
R_Gaia_err = R_Gaia_err_per * R_Gaia
#help(R_Gaia[0])
#print(KIC_Gaia[0])
#print(type(KIC_Gaia))
print(R_Gaia_err)

inv_r_gaia_K_MIST_err
---------------------
   1.1329750550674924
   0.3980318757106232
    0.599982638317747
   0.3656148490948969
  0.22786523565522532
    2.165530361313312
  0.44468518817024905
                  ...
    49.38366116265747
   0.8207903397948537
    8.380984118907424
     9.98327494418534
   369.12395747674947
    22.79847535484972
   23.205236370197742
Length = 12632 rows


In [89]:
#APOLLINAIRE file & file variables (b = background)
hdul_b = fits.open(r"/Users/amber3/Desktop/SSI/Selected_stars_final.fits")
hdu_b = hdul_b[0]
data_b = hdu_b.data
KIC_b = np.array(data_b[:,0,0])

In [90]:
#APAOKASC and background intersection and filtering
common_KIC, A_indices, b_indices = np.intersect1d(KIC_A, KIC_b, return_indices=True)
#print(np.shape(A_indices), np.shape(b_indices))
good_indices = np.where((Teff_A[A_indices] > 0) & (R_APO3[A_indices] > 0))
bad_indices = np.where(np.logical_or(Teff_A[A_indices] <= 0, R_APO3[A_indices] <= 0))
#bad_T_indices = np.where(Teff_A[A_indices] <= 0)
#bad_R_indices = np.where(R_APO3[A_indices] <= 0)
#print(Teff_A[A_indices[bad_T_indices]], R_APO3[A_indices[bad_R_indices]])
print(Teff_A[A_indices[bad_indices]], R_APO3[A_indices[bad_indices]])
print(bad_indices)
b_indices = b_indices[good_indices]
A_indices = A_indices[good_indices]
print(np.where(Teff_A[A_indices] < 0))
print(A_indices)
print(np.where(R_APO3[A_indices] < 0))

KIC_b = KIC_b[b_indices]
logg_final = Logg_A[A_indices]
logg_final_err = Logg_err[A_indices]
Teff_final = Teff_A[A_indices]
#print(Teff_final)
#print(np.where(Teff_final < 0)) 
log_Teff_final = np.log10(Teff_final)
Teff_final_err = Teff_err[A_indices]
Teff_err_log = ((Teff_final_err)/(log_Teff_final))/np.log(10) 
FeH_final = FeH_A[A_indices]
FeH_final_err = FeH_A_err[A_indices]

print(np.shape(logg_final), np.shape(KIC_b))
Amp_HV1 = np.array(data_b[b_indices,0,6])
Amp_HV1_err = np.array(data_b[b_indices,1,6])
Amp_HV2 = np.array(data_b[b_indices,0,9])
Amp_HV2_err = np.array(data_b[b_indices,1,9])
Freq_HV1 = np.array(data_b[b_indices,0,7])
Freq_HV1_err = np.array(data_b[b_indices,1,7])
Freq_HV2 = np.array(data_b[b_indices,0,10])
Freq_HV2_err = np.array(data_b[b_indices,1,10])
Evol_stage = np.array(data_b[b_indices,0,3])

[-9999.99  -9999.99  -9999.99   4805.594 -9999.99  -9999.99  -9999.99
 -9999.99  -9999.99  -9999.99 ] [-9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999.]
(array([ 318,  761,  794, 1898, 2156, 2211, 3250, 3297, 3401, 3605]),)
(array([], dtype=int64),)
[ 863 4166 4232 ... 5782 5784 5795]
(array([], dtype=int64),)
(3990,) (3990,)


In [91]:
#Gaia and background intersection and filtering
print(np.shape(KIC_b), np.shape(Amp_HV1))
common_KIC_G, A_indices_G, b_indices_G = np.intersect1d(KIC_Gaia, KIC_b, return_indices=True)
good_indices_G = np.where((R_Gaia[A_indices_G] > 0) & (R_Gaia_err[A_indices_G] > 0))
b_indices_G = b_indices_G[good_indices_G]
A_indices_G = A_indices_G[good_indices_G]

print(np.shape(A_indices_G), np.shape(b_indices_G))
KIC_final = KIC_Gaia[A_indices_G]
#use amps and freq data from previous data file 

logg_final_G = logg_final[b_indices_G]
logg_final_G_err = logg_final_err[b_indices_G]
Teff_final_G = Teff_final[b_indices_G]
#print(Teff_final)
#plt.plot(Teff_final_G, logg_final_G, '.')
#print(np.where(Teff_final_G < 0)) 
#print(np.shape(Teff_A), np.shape(Logg_A))
log_Teff_final_G = np.log10(Teff_final_G)
Teff_final_G_err = Teff_final_err[b_indices_G]
Teff_G_err_log = ((Teff_final_G_err)/(log_Teff_final_G))/np.log(10) 
FeH_final_G = FeH_final[b_indices_G]
FeH_final_G_err = FeH_final_err[b_indices_G]

Amp_HV1_G = Amp_HV1[b_indices_G]
Amp_HV1_G_err = Amp_HV1_err[b_indices_G]
Amp_HV2_G = Amp_HV2[b_indices_G]
Amp_HV2_G_err = Amp_HV2_err[b_indices_G]
Freq_HV1_G = Freq_HV1[b_indices_G]
Freq_HV1_G_err = Freq_HV1_err[b_indices_G]
Freq_HV2_G = Freq_HV2[b_indices_G]
Freq_HV2_G_err = Freq_HV2_err[b_indices_G]
Evol_stage_G = Evol_stage[b_indices_G]

(3990,) (3990,)
(3647,) (3647,)


In [92]:
R_APO3_final = R_APO3[A_indices]
R_APO3_final_err = R_APO3_err[A_indices]
R_Gaia_final = R_Gaia[A_indices_G] 
print(np.shape(R_Gaia_final))
R_Gaia_final_err = R_Gaia_err[A_indices_G]

rpw = 2

Amp_R_HV1_APO3 =  Amp_HV1 * R_APO3_final**rpw
#Amp_R_HV1_APO3_err = ((Amp_HV1_err/Amp_HV1) + (rpw*R_APO3_final_err/R_APO3_final)) * Amp_R_HV1_APO3
Amp_R_HV1_APO3_err_log = np.sqrt((Amp_HV1_err/Amp_HV1)**2 + (rpw*R_APO3_final_err/R_APO3_final)**2)/np.log(10)

Amp_R_HV2_APO3 =  Amp_HV2 * R_APO3_final**rpw
#Amp_R_HV2_APO3_err = ((Amp_HV2_err/Amp_HV2) + (rpw*R_APO3_final_err/R_APO3_final)) * Amp_R_HV2_APO3
Amp_R_HV2_APO3_err_log = np.sqrt((Amp_HV2_err/Amp_HV2)**2 + (rpw*R_APO3_final_err/R_APO3_final)**2)/np.log(10)

Amp_R_HV1_Gaia =  Amp_HV1_G * R_Gaia_final**rpw
Amp_R_HV1_Gaia_err_log = np.sqrt((Amp_HV1_G_err/Amp_HV1_G)**2 + (rpw*R_Gaia_final_err/R_Gaia_final)**2)/np.log(10)

Amp_R_HV2_Gaia =  Amp_HV2_G * R_Gaia_final**rpw
Amp_R_HV2_Gaia_err_log = np.sqrt((Amp_HV2_G_err/Amp_HV2_G)**2 + (rpw*R_Gaia_final_err/R_Gaia_final)**2)/np.log(10)

Amp_HV1_err_log = ((Amp_HV1_err)/(Amp_HV1))/np.log(10)
Amp_HV2_err_log = ((Amp_HV2_err)/(Amp_HV2))/np.log(10)
Freq_HV1_err_log = ((Freq_HV1_err)/(Freq_HV1))/np.log(10)
Freq_HV2_err_log = ((Freq_HV2_err)/(Freq_HV2))/np.log(10)

Amp_HV1_G_err_log = ((Amp_HV1_G_err)/(Amp_HV1_G))/np.log(10)
Amp_HV2_G_err_log = ((Amp_HV2_G_err)/(Amp_HV2_G))/np.log(10)
Freq_HV1_G_err_log = ((Freq_HV1_G_err)/(Freq_HV1_G))/np.log(10)
Freq_HV2_G_err_log = ((Freq_HV2_G_err)/(Freq_HV2_G))/np.log(10)

(3647,)


In [93]:
#APO3 rotated to atm params data

#%matplotlib inline
figure(figsize=(8,8), dpi=100)
def freq_curve_fit(X, a, b, c, d):
    x,y,z = X
    return a+b*x+c*y+z*d

def amp_curve_fit(X, a, b, c, d):
    x,y,z = X
    return a+b*x+c*y+z*d

def logg_fit(X, a, b):
    x = X
    return a + b*x
         
red_giant = np.where(Evol_stage == 1)
red_clump = np.where(Evol_stage == 2)
#print(Teff_final[red_giant])

mean_logg = np.mean(logg_final[red_giant])
mean_l_teff = np.mean(log_Teff_final[red_giant])
print(mean_logg, mean_l_teff) #neg value in log_Teff_final bc of runtime warning in Teff_final, printing nan

logg_opt, logg_cov = curve_fit(logg_fit, (np.log10(Teff_final[red_giant])-mean_l_teff), logg_final[red_giant]-mean_logg, sigma=(logg_final_err[red_giant]))
logg_perr = np.sqrt(np.diag(logg_cov))
logg_opt[1] = 18.614913/0.7

theta = ((np.pi/2) - np.arctan2(logg_opt[1], 1))
#rotated plots
x_prime = (log_Teff_final-mean_l_teff)*np.cos(theta) - ((logg_final-mean_logg)*np.sin(theta))
y_prime = (log_Teff_final-mean_l_teff)*np.sin(theta) + ((logg_final-mean_logg)*np.cos(theta))
coeffs = np.ones((4, 6, 2)) #switch 2 and 4?
coeffs_err = np.ones((4, 6, 2))

#frequency 1 red giants
F1_x_opt, F1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Freq_HV1[red_giant]), sigma=(Freq_HV1_err_log[red_giant]))
F1_x_perr = np.sqrt(np.diag(F1_x_cov))
#print(F1_x_opt)
#print(F1_x_perr)
coeffs[:,0,0] = F1_x_opt
#print(coeffs[:,0,0])
#print(F1_x_opt)
coeffs_err[:,0,0] = F1_x_perr

#frequency 1 red clump
F1_x_opt, F1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Freq_HV1[red_clump]), sigma=(Freq_HV1_err_log[red_clump]))
F1_x_perr = np.sqrt(np.diag(F1_x_cov))
#print(F1_x_opt)
#print(F1_x_perr)
coeffs[:,0,1] = F1_x_opt
coeffs_err[:,0,1] = F1_x_perr

#frequency 2 red giants
F2_x_opt, F2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Freq_HV2[red_giant]), sigma=(Freq_HV2_err_log[red_giant]))
F2_x_perr = np.sqrt(np.diag(F2_x_cov))
#in y direc -- dependancy very well constrained, x direc not really
#print(F2_x_opt)
#print(F2_x_perr)
coeffs[:,1,0] = F2_x_opt
coeffs_err[:,1,0] = F2_x_perr

#frequency 2 red clump
F2_x_opt, F2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Freq_HV2[red_clump]), sigma=(Freq_HV2_err_log[red_clump]))
F2_x_perr = np.sqrt(np.diag(F2_x_cov))
#fit more well defined in x direc now in clump, y direc worse
#red giant and red clump have opposite metallicity dependancy -- interesting
#print(F2_x_opt)
#print(F2_x_perr)
coeffs[:,1,1] = F2_x_opt
coeffs_err[:,1,1] = F2_x_perr

#print(red_giant)
#print(red_clump)

#amplitude 1 red giants
A1_x_opt, A1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_HV1[red_giant]), sigma=(Amp_HV1_err_log[red_giant]))
A1_x_perr = np.sqrt(np.diag(A1_x_cov))
print('A1_RG')
print(A1_x_opt)
print(A1_x_perr)
coeffs[:,2,0] = A1_x_opt
coeffs_err[:,2,0] = A1_x_perr

#amplitude 1 red clump
A1_x_opt, A1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_HV1[red_clump]), sigma=(Amp_HV1_err_log[red_clump]))
A1_x_perr = np.sqrt(np.diag(A1_x_cov))
print('A1_RC')
print(A1_x_opt)
print(A1_x_perr)
coeffs[:,2,1] = A1_x_opt
coeffs_err[:,2,1] = A1_x_perr

#amplitude 2 red giants
A2_x_opt, A2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_HV2[red_giant]), sigma=(Amp_HV2_err_log[red_giant]))
A2_x_perr = np.sqrt(np.diag(A2_x_cov))
print('A2_RG')
print(A2_x_opt)
print(A2_x_perr)
coeffs[:,3,0] = A2_x_opt
coeffs_err[:,3,0] = A2_x_perr

#amplitude 2 red clump
A2_x_opt, A2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_HV2[red_clump]), sigma=(Amp_HV2_err_log[red_clump]))
A2_x_perr = np.sqrt(np.diag(A2_x_cov))
print('A2_RC')
print(A2_x_opt)
print(A2_x_perr)
coeffs[:,3,1] = A2_x_opt
coeffs_err[:,3,1] = A2_x_perr

2.694704 3.6761162
A1_RG
[ 3.65773256 -9.75343131 -1.93054871 -0.0323873 ]
[0.00383415 0.52408347 0.01171772 0.01635012]
A1_RC
[  4.08869488 -19.31102444  -1.33376156  -0.80702151]
[0.01173537 0.54429958 0.03153781 0.02359385]
A2_RG
[ 2.9979083  -8.70223622 -2.06085666 -0.01834655]
[0.00425431 0.57752987 0.01283791 0.01783101]
A2_RC
[  3.42893417 -19.77150839  -1.44584805  -0.93976798]
[0.01267877 0.61411842 0.03396103 0.02631364]


<Figure size 800x800 with 0 Axes>

In [94]:
#fit in rotated atm params with respect to APO3 R corrected amps

#amplitude 1 red giants
A1_Rg_x_opt, A1_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_R_HV1_APO3[red_giant]), sigma=(Amp_R_HV1_APO3_err_log[red_giant]))
A1_Rg_x_perr = np.sqrt(np.diag(A1_Rg_x_cov))
coeffs[:,4,0] = A1_Rg_x_opt
coeffs_err[:,4,0] = A1_Rg_x_perr
print('A1_RG')
print(A1_Rg_x_opt)
print(A1_Rg_x_perr)

#amplitude 1 red clump
A1_Rc_x_opt, A1_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_R_HV1_APO3[red_clump]), sigma=(Amp_R_HV1_APO3_err_log[red_clump]))
A1_Rc_x_perr = np.sqrt(np.diag(A1_Rc_x_cov))
coeffs[:,4,1] = A1_Rc_x_opt
coeffs_err[:,4,1] = A1_Rc_x_perr
print('A1_RC')
print(A1_Rc_x_opt)
print(A1_Rc_x_perr)

#amplitude 2 red giants
A2_Rg_x_opt, A2_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_R_HV2_APO3[red_giant]), sigma=(Amp_R_HV2_APO3_err_log[red_giant]))
A2_Rg_x_perr = np.sqrt(np.diag(A2_Rg_x_cov))
coeffs[:,5,0] = A2_Rg_x_opt
coeffs_err[:,5,0] = A2_Rg_x_perr
print('A2_RG')
print(A2_Rg_x_opt)
print(A2_Rg_x_perr)

#amplitude 2 red clump
A2_Rc_x_opt, A2_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_R_HV2_APO3[red_clump]), sigma=(Amp_R_HV2_APO3_err_log[red_clump]))
A2_Rc_x_perr = np.sqrt(np.diag(A2_Rc_x_cov))
coeffs[:,5,1] = A2_Rc_x_opt
coeffs_err[:,5,1] = A2_Rc_x_perr
#print(np.shape(Amp_HV2_err_log[red_clump]))
#print(np.shape(x_prime[red_clump]))
print('A2_RC')
print(A2_Rc_x_opt)

A1_RG
[ 5.52396042 -4.74692705 -2.8822549   0.30129024]
[0.00598725 0.85877112 0.01901335 0.02671451]
A1_RC
[  6.16212644 -20.18203567  -1.41964945  -0.73017541]
[0.01280778 0.59450446 0.03438819 0.02577636]
A2_RG
[ 4.86349083 -3.65356419 -3.01714371  0.32267611]
[0.00657086 0.91094299 0.02015133 0.02810394]
A2_RC
[  5.51010671 -21.24625202  -1.54798993  -0.88402403]


In [95]:
#matched subsample of giants+clump of A1 and A2 with APO3 R
N_RC = np.size(red_clump)
log_Teff_final_RG = log_Teff_final[red_giant]
log_Teff_final_RC = log_Teff_final[red_clump]
logg_final_RG = logg_final[red_giant]
logg_final_RC = logg_final[red_clump]
FeH_final_RG = FeH_final[red_giant]
FeH_final_RC = FeH_final[red_clump]
#print(N_RC)
index_RC = ()
index_RG = ()
eps = 0.04
#print(type(index_RC))
#print(type(red_giant))
for i in range(0, N_RC): 
    dist = np.sqrt((7*(log_Teff_final_RG-log_Teff_final_RC[i]))**2 + (logg_final_RG-logg_final_RC[i])**2 + (3*(FeH_final_RG-FeH_final_RC[i]))**2)
    min_dist = (np.min(dist))
    if min_dist < eps:
        index = np.where(dist == min_dist)
        index_RC = index_RC + (i,)
        #index_RC = np.append(index_RC, i)
        index_RG = index_RG + (index[0],)
        #print(index[0])
#print(np.size(index_RC), np.size(index_RG))
#print(index_RG)
np.shape(index_RG)

iu = np.unique(index_RG, return_index=True)
iu = np.array(iu)[1,:]

Ni_RG = len(index_RG)
Ni_RC = len(index_RC)
N_iu = len(iu)

index_RG = np.reshape(index_RG, (Ni_RG,))
index_RC = np.reshape(index_RC, (Ni_RC,))

index_RC = index_RC[iu]
index_RG = index_RG[iu]

#amplitude 1 red giants and clump subsample
Amp_HV1_RG = Amp_R_HV1_APO3[red_giant]
Amp_HV1_RC = Amp_R_HV1_APO3[red_clump]

index_inliers = np.where((np.log10(Amp_HV1_RG[index_RG])-np.log10(Amp_HV1_RC[index_RC])) > (-2.4))
index_RC = index_RC[index_inliers]
index_RG = index_RG[index_inliers]

Amp_HV1_err_log_RG = Amp_R_HV1_APO3_err_log[red_giant]

A1_Rg_s_opt, A1_Rg_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]), np.log10(Amp_HV1_RG[index_RG]), sigma=(Amp_HV1_err_log_RG[index_RG]))
A1_Rg_s_perr = np.sqrt(np.diag(A1_Rg_s_cov))
print('A1_RG_s')
print(A1_Rg_s_opt)
print(A1_Rg_s_perr)

Amp_HV1_err_log_RC = Amp_R_HV1_APO3_err_log[red_clump]

A1_Rc_s_opt, A1_Rc_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]), np.log10(Amp_HV1_RC[index_RC]), sigma=(Amp_HV1_err_log_RC[index_RC]))
A1_Rc_s_perr = np.sqrt(np.diag(A1_Rc_s_cov))
print('A1_RC_s')
print(A1_Rc_s_opt)
print(A1_Rc_s_perr)

#amplitude 2 red giants and clump subsample
Amp_HV2_RG = Amp_R_HV2_APO3[red_giant]
Amp_HV2_RC = Amp_R_HV2_APO3[red_clump]

index_inliers = np.where((np.log10(Amp_HV2_RG[index_RG])-np.log10(Amp_HV2_RC[index_RC])) > (-2.4))
index_RC = index_RC[index_inliers]
index_RG = index_RG[index_inliers]

Amp_HV2_err_log_RG = Amp_R_HV2_APO3_err_log[red_giant]

A2_Rg_s_opt, A2_Rg_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]), np.log10(Amp_HV2_RG[index_RG]), sigma=(Amp_HV2_err_log_RG[index_RG]))
A2_Rg_s_perr = np.sqrt(np.diag(A2_Rg_s_cov))
print()
print('A2_RG_s')
print(A2_Rg_s_opt)
print(A2_Rg_s_perr)

Amp_HV2_err_log_RC = Amp_R_HV2_APO3_err_log[red_clump]

A2_Rc_s_opt, A2_Rc_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]), np.log10(Amp_HV2_RC[index_RC]), sigma=(Amp_HV2_err_log_RC[index_RC]))
A2_Rc_s_perr = np.sqrt(np.diag(A2_Rc_s_cov))
print('A2_RC_s')
print(A2_Rc_s_opt)
print(A2_Rc_s_perr)

A1_RG_s
[174.22053242 -45.1114973   -1.04312728  -0.40173659]
[23.14575886  6.43875812  0.27720497  0.21089998]
A1_RC_s
[ 62.76628384 -15.23451732  -0.18200265  -0.5126802 ]
[11.03760505  3.06245589  0.13152646  0.10203111]

A2_RG_s
[173.94069612 -45.12651968  -1.16907626  -0.33828067]
[22.90180773  6.36836906  0.27360429  0.20728784]
A2_RC_s
[ 65.01635807 -15.8681989   -0.40698154  -0.64524223]
[13.72335485  3.80487165  0.15648079  0.12048939]


In [96]:
#APO3 A1 ORTH FITTING

#GIANTS
x = [log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]]
sx = [Teff_err_log[index_RG], logg_final_err[index_RG], FeH_final_err[index_RG]]
A1_data = odr.RealData(x, [np.log10(Amp_HV1_RG[index_RG])], sx=sx, sy=[Amp_HV1_err_log_RG[index_RG]])
linear = odr.Model(odr_fit)
beta0 = [174.22, -45.11, -1.04, -0.40]
A1_odr = odr.ODR(A1_data, linear, beta0=beta0)
A1_output = A1_odr.run()
print('A1_Giants_APO3')
A1_output.pprint()

#CLUMP
x = [log_Teff_final_RG[index_RC], logg_final_RG[index_RC], FeH_final_RG[index_RC]]
sx = [Teff_err_log[index_RC], logg_final_err[index_RC], FeH_final_err[index_RC]]
A1_data = odr.RealData(x, [np.log10(Amp_HV1_RG[index_RC])], sx=sx, sy=[Amp_HV1_err_log_RG[index_RC]])
linear = odr.Model(odr_fit)
beta0 = [62.77, -15.23, -0.18, -0.51]
A1_odr = odr.ODR(A1_data, linear, beta0=beta0)
A1_output = A1_odr.run()
print()
print('A1_Clump_APO3')
A1_output.pprint()

A1_Giants_APO3
Beta: [263.41983242 -69.8158761   -0.43194421  -0.8408388 ]
Beta Std Error: [29.14441456  8.11054902  0.34124777  0.22433735]
Beta Covariance: [[ 7.61720076e+09 -2.11937954e+09  6.55334145e+07 -4.28737490e+07]
 [-2.11937954e+09  5.89909293e+08 -1.85595386e+07  1.19740657e+07]
 [ 6.55334145e+07 -1.85595386e+07  1.04429784e+06 -4.29876207e+05]
 [-4.28737490e+07  1.19740657e+07 -4.29876207e+05  4.51323457e+05]]
Residual Variance: 1.115103732644723e-07
Inverse Condition #: 0.00023088613751002358
Reason(s) for Halting:
  Iteration limit reached

A1_Clump_APO3
Beta: [ 82.27517967 -18.92601793  -2.64317436   0.38390341]
Beta Std Error: [11.50890333  3.15685573  0.13429293  0.14215536]
Beta Covariance: [[ 7.52681445e+07 -2.06359810e+07  2.25198308e+05 -1.20969851e+05]
 [-2.06359810e+07  5.66308130e+06 -6.88889672e+04  3.53443507e+04]
 [ 2.25198308e+05 -6.88889672e+04  1.02482475e+04 -2.77603472e+03]
 [-1.20969851e+05  3.53443507e+04 -2.77603472e+03  1.14833825e+04]]
Residual Var

In [97]:
#APO3 A2 ORTH FITTING

#GIANTS
x = [log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]]
sx = [Teff_err_log[index_RG], logg_final_err[index_RG], FeH_final_err[index_RG]]
A2_data = odr.RealData(x, [np.log10(Amp_HV2_RG[index_RG])], sx=sx, sy=[Amp_HV2_err_log_RG[index_RG]])
linear = odr.Model(odr_fit)
beta0 = [173.94, -45.13, -1.17, -0.34]
A2_odr = odr.ODR(A2_data, linear, beta0=beta0)
A2_output = A2_odr.run()
print('A2_Giants_APO3')
A2_output.pprint()

#CLUMP
x = [log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]]
sx = [Teff_err_log[index_RC], logg_final_err[index_RC], FeH_final_err[index_RC]]
A2_data = odr.RealData(x, [np.log10(Amp_HV2_RC[index_RC])], sx=sx, sy=[Amp_HV2_err_log_RC[index_RC]])
linear = odr.Model(odr_fit)
beta0 = [65.02, -15.87, -0.41, -0.65]
A2_odr = odr.ODR(A2_data, linear, beta0=beta0)
A2_output = A2_odr.run()
print()
print('A2_Clump_APO3')
A2_output.pprint()

A2_Giants_APO3
Beta: [252.83673341 -66.9632561   -0.64440914  -0.63514634]
Beta Std Error: [28.44150716  7.90780782  0.33871422  0.213198  ]
Beta Covariance: [[ 5.96735917e+09 -1.65878414e+09  4.91647203e+07 -2.91434933e+07]
 [-1.65878414e+09  4.61306079e+08 -1.39658022e+07  8.14205655e+06]
 [ 4.91647203e+07 -1.39658022e+07  8.46337974e+05 -2.95393166e+05]
 [-2.91434933e+07  8.14205655e+06 -2.95393166e+05  3.35307469e+05]]
Residual Variance: 1.355573390216774e-07
Inverse Condition #: 0.0002487422411537923
Reason(s) for Halting:
  Iteration limit reached

A2_Clump_APO3
Beta: [134.41783681 -34.92198321  -0.16859784  -0.98855672]
Beta Std Error: [16.31796065  4.50937205  0.17816913  0.13401358]
Beta Covariance: [[ 1.40755131e+09 -3.88870846e+08  8.38100136e+06 -7.98693135e+06]
 [-3.88870846e+08  1.07489145e+08 -2.39509626e+06  2.21772564e+06]
 [ 8.38100136e+06 -2.39509626e+06  1.67802097e+05 -6.27167837e+04]
 [-7.98693135e+06  2.21772564e+06 -6.27167837e+04  9.49358194e+04]]
Residual Vari

In [98]:
#GAIA data rotated, not radius corrected
#issues with theta_G...

#%matplotlib inline
figure(figsize=(8,8), dpi=100)
         
red_giant_G = np.where(Evol_stage_G == 1)
red_clump_G = np.where(Evol_stage_G == 2)
#print(Teff_final[red_giant_G])

mean_logg = np.mean(logg_final_G[red_giant_G])
mean_l_teff = np.mean(log_Teff_final_G[red_giant_G])
#print(mean_logg, mean_l_teff)
#print(mean_l_teff) #neg value in log_Teff_final bc of runtime warning in Teff_final, printing nan

logg_opt, logg_cov = curve_fit(logg_fit, (np.log10(Teff_final_G[red_giant_G])-mean_l_teff), logg_final_G[red_giant_G]-mean_logg, sigma=(logg_final_G_err[red_giant_G]))
logg_perr = np.sqrt(np.diag(logg_cov))
logg_opt[1] = 18.614913/0.7

theta_G = ((np.pi/2) - np.arctan2(logg_opt[1], 1))
#rotated plots
x_prime = ((np.log10(Teff_final_G)-mean_l_teff)*np.cos(theta_G)) - ((logg_final_G-mean_logg)*np.sin(theta_G))
y_prime = ((np.log10(Teff_final_G)-mean_l_teff)*np.sin(theta_G)) + ((logg_final_G-mean_logg)*np.cos(theta_G))
coeffs = np.ones((4, 6, 2)) #switch 2 and 4?
coeffs_err = np.ones((4, 6, 2))

#frequency 1 red giants
F1_x_opt, F1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant_G], y_prime[red_giant_G], FeH_final_G[red_giant_G]), np.log10(Freq_HV1_G[red_giant_G]), sigma=(Freq_HV1_G_err_log[red_giant_G]))
F1_x_perr = np.sqrt(np.diag(F1_x_cov))
#print(F1_x_opt)
#print(F1_x_perr)
coeffs[:,0,0] = F1_x_opt
#print(coeffs[:,0,0])
#print(F1_x_opt)
coeffs_err[:,0,0] = F1_x_perr

#frequency 1 red clump
F1_x_opt, F1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump_G], y_prime[red_clump_G], FeH_final_G[red_clump_G]), np.log10(Freq_HV1_G[red_clump_G]), sigma=(Freq_HV1_G_err_log[red_clump_G]))
F2_x_perr = np.sqrt(np.diag(F1_x_cov))
#print(F1_x_opt)
#print(F1_x_perr)
coeffs[:,0,1] = F1_x_opt
coeffs_err[:,0,1] = F1_x_perr

#frequency 2 red giants
F2_x_opt, F2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant_G], y_prime[red_giant_G], FeH_final_G[red_giant_G]), np.log10(Freq_HV2_G[red_giant_G]), sigma=(Freq_HV2_G_err_log[red_giant_G]))
F2_x_perr = np.sqrt(np.diag(F2_x_cov))
#in y direc -- dependancy very well constrained, x direc not really
#print(F2_x_opt)
#print(F2_x_perr)
coeffs[:,1,0] = F2_x_opt
coeffs_err[:,1,0] = F2_x_perr

#frequency 2 red clump
F2_x_opt, F2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump_G], y_prime[red_clump_G], FeH_final_G[red_clump_G]), np.log10(Freq_HV2_G[red_clump_G]), sigma=(Freq_HV2_G_err_log[red_clump_G]))
F2_x_perr = np.sqrt(np.diag(F2_x_cov))
#fit more well defined in x direc now in clump, y direc worse
#red giant and red clump have opposite metallicity dependancy -- interesting
#print(F2_x_opt)
#print(F2_x_perr)
coeffs[:,1,1] = F2_x_opt
coeffs_err[:,1,1] = F2_x_perr

#print(red_giant_G)
#print(red_clump_G)

#amplitude 1 red giants
A1_x_opt, A1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant_G], y_prime[red_giant_G], FeH_final_G[red_giant_G]), np.log10(Amp_HV1_G[red_giant_G]), sigma=(Amp_HV1_G_err_log[red_giant_G]))
A1_x_perr = np.sqrt(np.diag(A1_x_cov))
print('A1_RG')
print(A1_x_opt)
print(A1_x_perr)
coeffs[:,2,0] = A1_x_opt
coeffs_err[:,2,0] = A1_x_perr

#amplitude 1 red clump
A1_x_opt, A1_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump_G], y_prime[red_clump_G], FeH_final_G[red_clump_G]), np.log10(Amp_HV1_G[red_clump_G]), sigma=(Amp_HV1_G_err_log[red_clump_G]))
A1_x_perr = np.sqrt(np.diag(A1_x_cov))
print('A1_RC')
print(A1_x_opt)
print(A1_x_perr)
coeffs[:,2,1] = A1_x_opt
coeffs_err[:,2,1] = A1_x_perr

#amplitude 2 red giants
A2_x_opt, A2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant_G], y_prime[red_giant_G], FeH_final_G[red_giant_G]), np.log10(Amp_HV2_G[red_giant_G]), sigma=(Amp_HV2_G_err_log[red_giant_G]))
A2_x_perr = np.sqrt(np.diag(A2_x_cov))
print('A2_RG')
print(A2_x_opt)
print(A2_x_perr)
coeffs[:,3,0] = A2_x_opt
coeffs_err[:,3,0] = A2_x_perr

#amplitude 2 red clump
A2_x_opt, A2_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump_G], y_prime[red_clump_G], FeH_final_G[red_clump_G]), np.log10(Amp_HV2_G[red_clump_G]), sigma=(Amp_HV2_G_err_log[red_clump_G]))
A2_x_perr = np.sqrt(np.diag(A2_x_cov))
print('A2_RC')
print(A2_x_opt)
print(A2_x_perr)
coeffs[:,3,1] = A2_x_opt
coeffs_err[:,3,1] = A2_x_perr

A1_RG
[  3.66359347 -10.37443534  -1.93290392  -0.0452274 ]
[0.00411703 0.55782711 0.0124876  0.01731535]
A1_RC
[  4.08478907 -18.72387079  -1.33715895  -0.78681168]
[0.01273339 0.5838005  0.03424734 0.02541736]
A2_RG
[ 3.00390034 -9.1042833  -2.06108398 -0.02829878]
[0.00458347 0.61476158 0.0137264  0.01895025]
A2_RC
[  3.43563765 -19.49635632  -1.42323392  -0.93882159]
[0.01408491 0.66979898 0.03747503 0.02911494]


<Figure size 800x800 with 0 Axes>

In [99]:
#fit in rotated atm params with respect to Gaia R corrected amps
#with sigma

#amplitude 1 red giants
A1_Rg_x_opt, A1_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant_G], y_prime[red_giant_G], FeH_final_G[red_giant_G]), np.log10(Amp_R_HV1_Gaia[red_giant_G]), sigma=(Amp_R_HV1_Gaia_err_log[red_giant_G]))
A1_Rg_x_perr = np.sqrt(np.diag(A1_Rg_x_cov))
coeffs[:,4,0] = A1_Rg_x_opt
coeffs_err[:,4,0] = A1_Rg_x_perr
print('A1_RG')
print(A1_Rg_x_opt)
print(A1_Rg_x_perr)

#amplitude 1 red clump
A1_Rc_x_opt, A1_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump_G], y_prime[red_clump_G], FeH_final_G[red_clump_G]), np.log10(Amp_R_HV1_Gaia[red_clump_G]), sigma=(Amp_R_HV1_Gaia_err_log[red_clump_G]))
A1_Rc_x_perr = np.sqrt(np.diag(A1_Rc_x_cov))
coeffs[:,4,1] = A1_Rc_x_opt
coeffs_err[:,4,1] = A1_Rc_x_perr
print('A1_RC')
print(A1_Rc_x_opt)
print(A1_Rc_x_perr)

#amplitude 2 red giants
A2_Rg_x_opt, A2_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant_G], y_prime[red_giant_G], FeH_final_G[red_giant_G]), np.log10(Amp_R_HV2_Gaia[red_giant_G]), sigma=(Amp_R_HV2_Gaia_err_log[red_giant_G]))
A2_Rg_x_perr = np.sqrt(np.diag(A2_Rg_x_cov))
coeffs[:,5,0] = A2_Rg_x_opt
coeffs_err[:,5,0] = A2_Rg_x_perr
print('A2_RG')
print(A2_Rg_x_opt)
print(A2_Rg_x_perr)

#amplitude 2 red clump
A2_Rc_x_opt, A2_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump_G], y_prime[red_clump_G], FeH_final_G[red_clump_G]), np.log10(Amp_R_HV2_Gaia[red_clump_G]), sigma=(Amp_R_HV2_Gaia_err_log[red_clump_G]))
A2_Rc_x_perr = np.sqrt(np.diag(A2_Rc_x_cov))
coeffs[:,5,1] = A2_Rc_x_opt
coeffs_err[:,5,1] = A2_Rc_x_perr
#print(np.shape(Amp_HV2_err_log[red_clump_G]))
#print(np.shape(x_prime[red_clump_G]))
print('A2_RC')
print(A2_Rc_x_opt)
print(A2_Rc_x_perr)

A1_RG
[ 5.53873895 -5.95065951 -2.91025008  0.27860142]
[0.00613588 0.91694283 0.01904004 0.02772421]
A1_RC
[  6.15140929 -19.84033252  -1.45030676  -0.69386675]
[0.01434253 0.64915916 0.0392123  0.0281248 ]
A2_RG
[ 4.8708682  -5.557648   -3.02994192  0.28377527]
[0.00675918 0.97361474 0.02068394 0.02950886]
A2_RC
[  5.50260215 -20.74917482  -1.55691089  -0.85176529]
[0.01609653 0.74820355 0.04357745 0.03240875]


In [100]:
#matched subsample of giants+clump of A1 and A2 with Gaia R
N_RC = np.size(red_clump_G)
log_Teff_final_RG = log_Teff_final_G[red_giant_G]
log_Teff_final_RC = log_Teff_final_G[red_clump_G]
logg_final_RG = logg_final_G[red_giant_G]
logg_final_RC = logg_final_G[red_clump_G]
FeH_final_RG = FeH_final_G[red_giant_G]
FeH_final_RC = FeH_final_G[red_clump_G]
#print(N_RC)
index_RC = ()
index_RG = ()
eps = 0.04
#print(type(index_RC))
#print(type(red_giant_G))
for i in range(0, N_RC): 
    dist = np.sqrt((7*(log_Teff_final_RG-log_Teff_final_RC[i]))**2 + (logg_final_RG-logg_final_RC[i])**2 + (3*(FeH_final_RG-FeH_final_RC[i]))**2)
    min_dist = (np.min(dist))
    if min_dist < eps:
        index = np.where(dist == min_dist)
        index_RC = index_RC + (i,)
        #index_RC = np.append(index_RC, i)
        index_RG = index_RG + (index[0],)
        #print(index[0])
#print(np.size(index_RC), np.size(index_RG))
#print(index_RG)
np.shape(index_RG)

iu = np.unique(index_RG, return_index=True)
iu = np.array(iu)[1,:]

Ni_RG = len(index_RG)
Ni_RC = len(index_RC)
N_iu = len(iu)

index_RG = np.reshape(index_RG, (Ni_RG,))
index_RC = np.reshape(index_RC, (Ni_RC,))

index_RC = index_RC[iu]
index_RG = index_RG[iu]

#amplitude 1 red giants and clump subsample
Amp_HV1_RG = Amp_R_HV1_Gaia[red_giant_G]
Amp_HV1_RC = Amp_R_HV1_Gaia[red_clump_G]

index_inliers = np.where((np.log10(Amp_HV1_RG[index_RG])-np.log10(Amp_HV1_RC[index_RC])) > (-2.4))
index_RC = index_RC[index_inliers]
index_RG = index_RG[index_inliers]

Amp_HV1_err_log_RG = Amp_HV1_G_err_log[red_giant_G]

A1_Rg_s_opt, A1_Rg_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]), np.log10(Amp_HV1_RG[index_RG]), sigma=(Amp_HV1_err_log_RG[index_RG]))
A1_Rg_s_perr = np.sqrt(np.diag(A1_Rg_s_cov))
print('A1_RG_s')
print(A1_Rg_s_opt)
print(A1_Rg_s_perr)

Amp_HV1_err_log_RC = Amp_HV1_G_err_log[red_clump_G]

A1_Rc_s_opt, A1_Rc_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]), np.log10(Amp_HV1_RC[index_RC]), sigma=(Amp_HV1_err_log_RC[index_RC]))
A1_Rc_s_perr = np.sqrt(np.diag(A1_Rc_s_cov))
print('A1_RC_s')
print(A1_Rc_s_opt)
print(A1_Rc_s_perr)

#amplitude 2 red giants and clump subsample
Amp_HV2_RG = Amp_R_HV2_Gaia[red_giant_G]
Amp_HV2_RC = Amp_R_HV2_Gaia[red_clump_G]

index_inliers = np.where((np.log10(Amp_HV2_RG[index_RG])-np.log10(Amp_HV2_RC[index_RC])) > (-2.4))
index_RC = index_RC[index_inliers]
index_RG = index_RG[index_inliers]

Amp_HV2_err_log_RG = Amp_HV2_G_err_log[red_giant_G]

A2_Rg_s_opt, A2_Rg_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]), np.log10(Amp_HV2_RG[index_RG]), sigma=(Amp_HV2_err_log_RG[index_RG]))
A2_Rg_s_perr = np.sqrt(np.diag(A2_Rg_s_cov))
print()
print('A2_RG_s')
print(A2_Rg_s_opt)
print(A2_Rg_s_perr)

Amp_HV2_err_log_RC = Amp_HV2_G_err_log[red_clump_G]

A2_Rc_s_opt, A2_Rc_s_cov = curve_fit(amp_curve_fit, (log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]), np.log10(Amp_HV2_RC[index_RC]), sigma=(Amp_HV2_err_log_RC[index_RC]))
A2_Rc_s_perr = np.sqrt(np.diag(A2_Rc_s_cov))
print('A2_RC_s')
print(A2_Rc_s_opt)
print(A2_Rc_s_perr)

A1_RG_s
[183.71571653 -47.91834336  -0.72841156  -0.52330306]
[25.89758555  7.19738104  0.30700354  0.23238951]
A1_RC_s
[ 62.1514241  -15.00819939  -0.2695965   -0.52944989]
[11.66784067  3.23924806  0.15038853  0.10873669]

A2_RG_s
[176.48620073 -46.01244424  -0.8914465   -0.4247619 ]
[25.47381592  7.07809962  0.30773624  0.22383936]
A2_RC_s
[ 59.99126726 -14.43890511  -0.50144413  -0.63372235]
[14.66485416  4.06627636  0.17601443  0.12952737]


In [101]:
#GAIA A1 ORTH FITTING

def odr_fit(A, X):
    x,y,z = X
    return A[0]+A[1]*x+A[2]*y+A[3]*z

#GIANTS
x = [log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]]
sx = [Teff_G_err_log[index_RG], logg_final_G_err[index_RG], FeH_final_G_err[index_RG]]
A1_data = odr.RealData(x, [np.log10(Amp_HV1_RG[index_RG])], sx=sx, sy=[Amp_HV1_err_log_RG[index_RG]])
linear = odr.Model(odr_fit)
beta0 = [183.7, -48.0, -0.73, -0.52]
A1_odr = odr.ODR(A1_data, linear, beta0=beta0)
A1_output = A1_odr.run()
print('A1_Giants_Gaia')
A1_output.pprint()

#CLUMP
x = [log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]]
sx = [Teff_G_err_log[index_RC], logg_final_G_err[index_RC], FeH_final_G_err[index_RC]]
A1_data = odr.RealData(x, [np.log10(Amp_HV1_RC[index_RC])], sx=sx, sy=[Amp_HV1_err_log_RC[index_RC]])
linear = odr.Model(odr_fit)
beta0 = [62.15, -15.01, -0.27, -0.53]
A1_odr = odr.ODR(A1_data, linear, beta0=beta0)
A1_output = A1_odr.run()
print()
print('A1_Clump_Gaia')
A1_output.pprint()

#figure(figsize=(8,8), dpi=100)

#fit_diff_orth = np.log10(Amp_HV1_RG[index_RG])-log_Teff_final_RG[index_RG]*(-7.76554532e+01)-2.91513920e+02-logg_final_RG[index_RG]*(-1.67583097e-01)-FeH_final_RG[index_RG]*(-1.15637010e+00)
#fit_diff_ls = np.log10(Amp_HV1_RG[index_RG])-log_Teff_final_RG[index_RG]*(A1_Rg_s_opt[1])-logg_final_RG[index_RG]*(A1_Rg_s_opt[2])-FeH_final_RG[index_RG]*(A1_Rg_s_opt[3])-A1_Rg_s_opt[0]
#plt.plot(log_Teff_final_RG[index_RG], (np.log10(Amp_HV1_RG[index_RG]))-log_Teff_final_RG[index_RG]*(-7.76554532e+01)-2.91513920e+02-logg_final_RG[index_RG]*(-1.67583097e-01)-FeH_final_RG[index_RG]*(-1.15637010e+00), 'b.') #orth distance fitting
#plt.plot(log_Teff_final_RG[index_RG], log_Teff_final_RG[index_RG]*(-7.76554532e+01)+2.91513920e+02)
#plt.plot(log_Teff_final_RG[index_RG], (np.log10(Amp_HV1_RG[index_RG]))-log_Teff_final_RG[index_RG]*(A1_Rg_s_opt[1])-logg_final_RG[index_RG]*(A1_Rg_s_opt[2])-FeH_final_RG[index_RG]*(A1_Rg_s_opt[3])-A1_Rg_s_opt[0], 'r.') #least square fitting
#print(np.mean(fit_diff_orth))
#print(np.std(fit_diff_orth))

#print(np.mean(fit_diff_ls))
#print(np.std(fit_diff_ls))

A1_Giants_Gaia
Beta: [ 2.91513920e+02 -7.76554532e+01 -1.67583097e-01 -1.15637010e+00]
Beta Std Error: [36.45594694 10.12382877  0.40310871  0.27912219]
Beta Covariance: [[ 1.11487462e+10 -3.09543537e+09  8.61736972e+07 -6.54029663e+07]
 [-3.09543537e+09  8.59763184e+08 -2.43971542e+07  1.82026465e+07]
 [ 8.61736972e+07 -2.43971542e+07  1.36311852e+06 -5.61580089e+05]
 [-6.54029663e+07  1.82026465e+07 -5.61580089e+05  6.53548757e+05]]
Residual Variance: 1.192094648334626e-07
Inverse Condition #: 0.00023062703826252277
Reason(s) for Halting:
  Iteration limit reached

A1_Clump_Gaia
Beta: [ 72.46090622 -17.78448125  -0.30928306  -0.56045233]
Beta Std Error: [7.74916245 2.14511204 0.14268897 0.09163023]
Beta Covariance: [[ 1.32169035e+08 -3.65546666e+07  8.47627469e+05 -7.21916925e+05]
 [-3.65546666e+07  1.01279173e+07 -2.60874175e+05  2.02261513e+05]
 [ 8.47627469e+05 -2.60874175e+05  4.48126872e+04 -8.06580514e+03]
 [-7.21916925e+05  2.02261513e+05 -8.06580514e+03  1.84798186e+04]]
Resi

In [102]:
#GAIA A2 ORTH FITTING

#GIANTS
x = [log_Teff_final_RG[index_RG], logg_final_RG[index_RG], FeH_final_RG[index_RG]]
sx = [Teff_G_err_log[index_RG], logg_final_G_err[index_RG], FeH_final_G_err[index_RG]]
A2_data = odr.RealData(x, [np.log10(Amp_HV2_RG[index_RG])], sx=sx, sy=[Amp_HV2_err_log_RG[index_RG]])
linear = odr.Model(odr_fit)
beta0 = [176.49, -46.01, -0.89, -0.42]
A2_odr = odr.ODR(A2_data, linear, beta0=beta0)
A2_output = A2_odr.run()
print('A2_Giants_Gaia')
A2_output.pprint()

#CLUMP
x = [log_Teff_final_RC[index_RC], logg_final_RC[index_RC], FeH_final_RC[index_RC]]
sx = [Teff_G_err_log[index_RC], logg_final_G_err[index_RC], FeH_final_G_err[index_RC]]
A2_data = odr.RealData(x, [np.log10(Amp_HV2_RC[index_RC])], sx=sx, sy=[Amp_HV2_err_log_RC[index_RC]])
linear = odr.Model(odr_fit)
beta0 = [59.99, -14.44, -0.50, -0.63]
A2_odr = odr.ODR(A2_data, linear, beta0=beta0)
A2_output = A2_odr.run()
print()
print('A2_Clump_Gaia')
A2_output.pprint()

A2_Giants_Gaia
Beta: [198.59302377 -51.90827333  -1.06865487  -0.54100085]
Beta Std Error: [21.11271535  5.85332084  0.30130688  0.19812214]
Beta Covariance: [[ 1.99890462e+09 -5.53922688e+08  1.39010293e+07 -1.05349398e+07]
 [-5.53922688e+08  1.53641502e+08 -4.06198088e+06  2.93851584e+06]
 [ 1.39010293e+07 -4.06198088e+06  4.07119561e+05 -9.78175747e+04]
 [-1.05349398e+07  2.93851584e+06 -9.78175747e+04  1.76023195e+05]]
Residual Variance: 2.2299550755320297e-07
Inverse Condition #: 0.0003505381432355575
Reason(s) for Halting:
  Iteration limit reached

A2_Clump_Gaia
Beta: [ 98.4510676  -24.96500459  -0.4143451   -0.8061231 ]
Beta Std Error: [11.69616802  3.23046985  0.1746714   0.11640136]
Beta Covariance: [[ 4.03548716e+08 -1.11396498e+08  2.26877829e+06 -2.13835401e+06]
 [-1.11396498e+08  3.07850708e+07 -6.78149368e+05  5.95360313e+05]
 [ 2.26877829e+06 -6.78149368e+05  9.00020464e+04 -1.87426532e+04]
 [-2.13835401e+06  5.95360313e+05 -1.87426532e+04  3.99691461e+04]]
Residual Var